In [2]:
# Libraries
import pandas as pd
import numpy as np 

pd.set_option('display.max_columns', None)
from datetime import datetime


In [3]:
# Functions 
def standardize_colnames(df):
    """
    Formats column names of a dataframe: removes capital letters and spaces
    """
    dict = {}
    for col_name in df.columns:
        key = col_name 
        value = col_name.lower().replace(' ', '_')
        dict.update({key : value})
    df.rename(columns = dict, inplace= True)
    return df

def check_dayoff(x):
    """ 
    Returns 1 if a day is a day-off and 0 if not, based on weekday number 
    """
    no = x.weekday()
    if no <5:
        return '0'
    else:
        return '1'

def assign_q(x):
    """
    Returns quartal based on a month number 
    """
    if x in [1,2,3]:
        return 'Q1'
    elif x in [4,5,6]:
        return 'Q2'
    elif x in [7,8,9]:
        return 'Q3'
    elif x in [10,11,12]:
        return 'Q4'



## Original dataset transformation

In [4]:
# Loading data 
df = pd.read_csv('supermarket_sales_kaggle.csv') 
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 17 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Invoice ID               1000 non-null   object 
 1   Branch                   1000 non-null   object 
 2   City                     1000 non-null   object 
 3   Customer type            1000 non-null   object 
 4   Gender                   1000 non-null   object 
 5   Product line             1000 non-null   object 
 6   Unit price               1000 non-null   float64
 7   Quantity                 1000 non-null   int64  
 8   Tax 5%                   1000 non-null   float64
 9   Total                    1000 non-null   float64
 10  Date                     1000 non-null   object 
 11  Time                     1000 non-null   object 
 12  Payment                  1000 non-null   object 
 13  cogs                     1000 non-null   float64
 14  gross margin percentage  

In [5]:
df.head(5)

,Invoice ID,Branch,City,Customer type,Gender,Product line,Unit price,Quantity,Tax 5%,Total,Date,Time,Payment,cogs,gross margin percentage,gross income,Rating
0,750-67-8428,A,Yangon,Member,Female,Health and beauty,74.69,7,26.1415,548.9715,1/5/2019,13:08,Ewallet,522.83,4.761905,26.1415,9.1
1,226-31-3081,C,Naypyitaw,Normal,Female,Electronic accessories,15.28,5,3.8200,80.2200,3/8/2019,10:29,Cash,76.40,4.761905,3.8200,9.6
2,631-41-3108,A,Yangon,Normal,Male,Home and lifestyle,46.33,7,16.2155,340.5255,3/3/2019,13:23,Credit card,324.31,4.761905,16.2155,7.4
3,123-19-1176,A,Yangon,Member,Male,Health and beauty,58.22,8,23.2880,489.0480,1/27/2019,20:33,Ewallet,465.76,4.761905,23.2880,8.4
4,373-73-7910,A,Yangon,Normal,Male,Sports and travel,86.31,7,30.2085,634.3785,2/8/2019,10:37,Ewallet,604.17,4.761905,30.2085,5.3


In [6]:
# Column names transformation (it's optional but for me it's more convinient to adress columns without capital letters or spaces in names)
df = standardize_colnames(df)
df.columns

Index(['invoice_id', 'branch', 'city', 'customer_type', 'gender',
       'product_line', 'unit_price', 'quantity', 'tax_5%', 'total', 'date',
       'time', 'payment', 'cogs', 'gross_margin_percentage', 'gross_income',
       'rating'],
      dtype='object')

In [7]:
df.nunique()
# Invoice id is a useless col, none information can be extracted

invoice_id                 1000
branch                        3
city                          3
customer_type                 2
gender                        2
product_line                  6
unit_price                  943
quantity                     10
tax_5%                      990
total                       990
date                         89
time                        506
payment                       3
cogs                        990
gross_margin_percentage       1
gross_income                990
rating                       61
dtype: int64

In [8]:
# Adding desired columns
df = df.assign(
    formatted_date = pd.to_datetime(df['date']),
    purchase_hour = pd.to_datetime(df['time'], format = '%H:%M').dt.hour,
    month_number = lambda x: x['formatted_date'].dt.month,
    month_name = lambda x: x['formatted_date'].dt.month_name(locale = 'English'),
    week_no = lambda x: x['formatted_date'].dt.isocalendar().week,
    is_weekend = lambda x: x['formatted_date'].apply(check_dayoff),
    quartal = lambda x: x['month_number'].apply(assign_q),
    year = lambda x: x['formatted_date'].dt.year, 
    )

df = df.drop( columns = ['date', 'time', 'invoice_id'])
df.head(3)

,branch,city,customer_type,gender,product_line,unit_price,quantity,tax_5%,total,payment,cogs,gross_margin_percentage,gross_income,rating,formatted_date,purchase_hour,month_number,month_name,week_no,is_weekend,quartal,year
0,A,Yangon,Member,Female,Health and beauty,74.69,7,26.1415,548.9715,Ewallet,522.83,4.761905,26.1415,9.1,2019-01-05,13,1,January,1,1,Q1,2019
1,C,Naypyitaw,Normal,Female,Electronic accessories,15.28,5,3.8200,80.2200,Cash,76.40,4.761905,3.8200,9.6,2019-03-08,10,3,March,10,0,Q1,2019
2,A,Yangon,Normal,Male,Home and lifestyle,46.33,7,16.2155,340.5255,Credit card,324.31,4.761905,16.2155,7.4,2019-03-03,13,3,March,9,1,Q1,2019


In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 22 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   branch                   1000 non-null   object        
 1   city                     1000 non-null   object        
 2   customer_type            1000 non-null   object        
 3   gender                   1000 non-null   object        
 4   product_line             1000 non-null   object        
 5   unit_price               1000 non-null   float64       
 6   quantity                 1000 non-null   int64         
 7   tax_5%                   1000 non-null   float64       
 8   total                    1000 non-null   float64       
 9   payment                  1000 non-null   object        
 10  cogs                     1000 non-null   float64       
 11  gross_margin_percentage  1000 non-null   float64       
 12  gross_income             1000 non-n

In [10]:
df = df.sort_values(['branch','formatted_date', 'purchase_hour'])
df.to_csv('sales_opt.csv', index=False)

## Create the second dataframe


In [11]:
# This code can be copied and pasted into Power BI. Path to modified dataframe should be provided

import pandas as pd

filepath = ""

df = pd.read_csv(filepath)

df1 = df.groupby(['year', 'quartal', 'week_no', 'is_weekend']).agg(
    sales = ('total', 'sum'),
    items_sold = ('quantity', 'sum'),
    number_of_days = ('formatted_date', 'nunique'),
    number_of_transactions = ('total', 'count')
).sort_values(['year', 'week_no']).reset_index().dropna()

df1 = df1.query("items_sold>0")

df1 = df1.assign(
    avg_daily_sales = df1.sales/df1.number_of_days,
    avg_daily_items_sold = df1.items_sold/df1.number_of_days, 
    avg_daily_num_transaction = df1.number_of_transactions/df1.number_of_days,
    avg_bill = df1['sales']/df1['number_of_transactions']
)

weekno = df1.week_no.unique().tolist()

# Remove weeks which aren't fully observable (there's data only for Mon to Sat, for example)
res = []
for i in weekno:
    subdf = df1[df1['week_no'] == i]
    #print(subdf.number_of_days.sum())
    #print(subdf.shape)
    if subdf.number_of_days.sum() != 7:
        for i in range(subdf.shape[0]):
            res.append('False')
    else:
        for i in range(subdf.shape[0]):
            res.append('True')

df1['full_week'] = res

df1 = df1.query("full_week == 'True'").drop(columns = 'full_week')
df1 = pd.merge(df1, df[['week_no', 'month_name']], on = 'week_no', how = 'left')

df1



,year,quartal,week_no,is_weekend,sales,items_sold,number_of_days,number_of_transactions,avg_daily_sales,avg_daily_items_sold,avg_daily_num_transaction,avg_bill,month_name
0,2019,Q1,2,0,16825.2315,301,5,52,3365.04630,60.2,10.4,323.562144,January
1,2019,Q1,2,0,16825.2315,301,5,52,3365.04630,60.2,10.4,323.562144,January
2,2019,Q1,2,0,16825.2315,301,5,52,3365.04630,60.2,10.4,323.562144,January
3,2019,Q1,2,0,16825.2315,301,5,52,3365.04630,60.2,10.4,323.562144,January
4,2019,Q1,2,0,16825.2315,301,5,52,3365.04630,60.2,10.4,323.562144,January
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1763,2019,Q1,12,1,7572.5055,124,2,22,3786.25275,62.0,11.0,344.204795,March
1764,2019,Q1,12,1,7572.5055,124,2,22,3786.25275,62.0,11.0,344.204795,March
1765,2019,Q1,12,1,7572.5055,124,2,22,3786.25275,62.0,11.0,344.204795,March
1766,2019,Q1,12,1,7572.5055,124,2,22,3786.25275,62.0,11.0,344.204795,March


In [12]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1768 entries, 0 to 1767
Data columns (total 13 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   year                       1768 non-null   int64  
 1   quartal                    1768 non-null   object 
 2   week_no                    1768 non-null   int64  
 3   is_weekend                 1768 non-null   int64  
 4   sales                      1768 non-null   float64
 5   items_sold                 1768 non-null   int64  
 6   number_of_days             1768 non-null   int64  
 7   number_of_transactions     1768 non-null   int64  
 8   avg_daily_sales            1768 non-null   float64
 9   avg_daily_items_sold       1768 non-null   float64
 10  avg_daily_num_transaction  1768 non-null   float64
 11  avg_bill                   1768 non-null   float64
 12  month_name                 1768 non-null   object 
dtypes: float64(5), int64(6), object(2)
memory usage: